# Nanodomain Collision Model – Extended Metrics
This notebook implements parameters 2,3,5.

In [1]:

import numpy as np
import matplotlib.pyplot as plt

# Basic simulation parameters
R = 1.0
dt = 0.01
steps = 2000

def brownian_step(pos, D):
    return pos + np.sqrt(2*D*dt)*np.random.randn(*pos.shape)

def simulate(n_rh=1, n_g=50, n_pde=30, D_rh=0.005, D_g=0.01, D_pde=0.008, r_int=0.05):
    # positions
    rh = np.random.uniform(-R,R,(n_rh,2))
    g = np.random.uniform(-R,R,(n_g,2))
    pde = np.random.uniform(-R,R,(n_pde,2))

    g_active = np.zeros(n_g, dtype=bool)
    pde_active = np.zeros(n_pde, dtype=bool)

    PDE_count = []
    encounter_counts = {"rh_g":0,"g_pde":0}
    effective_counts = {"rh_g":0,"g_pde":0}

    for t in range(steps):
        # diffusion
        rh[:] = brownian_step(rh, D_rh)
        g[:] = brownian_step(g, D_g)
        pde[:] = brownian_step(pde, D_pde)

        # reflect boundaries
        for arr in (rh,g,pde):
            out = np.linalg.norm(arr,axis=1)>R
            arr[out] *= R/np.linalg.norm(arr[out],axis=1)[:,None]

        # Rh-G interactions
        for i, r in enumerate(rh):
            dist = np.linalg.norm(g - r, axis=1)
            hits = dist < r_int
            encounter_counts["rh_g"] += hits.sum()
            newly = hits & (~g_active)
            effective_counts["rh_g"] += newly.sum()
            g_active[newly] = True

        # G-PDE interactions
        for i, gg in enumerate(g[g_active]):
            dist = np.linalg.norm(pde - gg, axis=1)
            hits = dist < r_int
            encounter_counts["g_pde"] += hits.sum()
            newly = hits & (~pde_active)
            effective_counts["g_pde"] += newly.sum()
            pde_active[newly] = True

        PDE_count.append(pde_active.sum())

    return {
        "PDE_count": np.array(PDE_count),
        "encounters": encounter_counts,
        "effective": effective_counts
    }

# Curve input-output
def input_output_curve(rh_list):
    peaks=[]
    for n in rh_list:
        out = simulate(n_rh=n)
        peaks.append(out["PDE_count"].max())
    return rh_list, peaks

print("Notebook loaded.")


Notebook loaded.
